# Packages

In [ ]:
import numpy as np
from numpy.linalg import inv
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc
rc('text', usetex=False)
import time
from numba import jit
import pickle
from bisect import bisect_left

# Used functions 

In [ ]:
# supplementary function
@jit(nopython=True)
def eta(p):
    # define eta(p), here we use eta(p)=[1,p]
    return np.array([1,p])

def get_d_l(p, theta_S, m):
    '''
    This function is used to get d_l or d_u
    '''
    d_l = [np.dot(eta(p),theta_S[k-1]) for k in range(1,m+2)]
    return d_l

def get_y_l(d, H, sigma_m):
    '''
    This function is used to get y_l
    '''
    y = round(sum(d) + 4 * np.sqrt(H) * sigma_m)
    return y
        
def get_V(T, V_TYPE):
    if V_TYPE == "LOG":
        return np.log2(T)
    elif V_TYPE == '0.2':
        return np.power(T, 0.2)
    elif V_TYPE == '0.5':
        return np.power(T, 0.5)
    else:
        raise ValueError(f"Error: V_TYPE is not correct. Supported V_TYPEs are 'LOG', '0.2' and '0.5'.")

@jit(nopython=True)
def get_T_2(T):
    # epochs of solving DP
    T_2=[2**i for i in range(1,round(np.log2(T))+1)] 
    return T_2

def get_S(m, H, UNI_ALOC):
    if UNI_ALOC:
        # uniform allocation
        S = uniform_allocation(m=m,H=H)
    else:
        S = list(range(1,m+2))
        S.append(H+1)
    return S

def get_theta_S(a, b, m, S):
    '''
    theta = (a,b)
    S is the re-pricing point list
    m is the number of re-price points
    '''
    theta_S=[[sum((a[i-1] for i in range(S[k-1],S[k]))),sum((b[i-1] for i in range(S[k-1],S[k])))] for k in range(1,m+2)]
    return theta_S
    
def get_sigma_S(sigma, m, S):
    '''
    sigma is the variance of random part demand
    S is the re-pricing point list
    m is the number of re-price points
    '''
    sigma_S=[np.sqrt(sum(sigma[i-1]**2 for i in range(S[k-1],S[k]))) for k in range(1,m+2)]
    return sigma_S


def get_noise_S(noise_all, m, T, S):
    '''
    noise_all is a 2-d dict
    epoch(T) - re-pricing period(m)
    '''
    noise_S_all = [[sum(noise_all[t][i-1] for i in range(S[k-1],S[k])) for k in range(1,m+2)] for t in range(T)]
    return noise_S_all


def minus(x, y):
    try:
        len(x)
        regret = [x[i]-y[i] for i in range(len(x))]
    except TypeError:
        regret = x - y
    return regret


@jit(nopython=True)
def proj_idx(x, seq):    
    '''
    Return the location of x in seq (how many numbers in seq smaller than x)
    '''
    return np.searchsorted(seq, np.minimum(x,seq[-1]))

@jit(nopython=True)
def proj(x,seq):
    '''
    Return the biggest number in seq that smaller than x
    '''
    return seq[proj_idx(x, seq)] 

def mean_row(data):
    mean_values = np.mean(data, axis=0)
    mean_values.tolist()
    return mean_values
    
@jit(nopython=True)
def G(x):
    '''
    Holding and backlogging cost
    '''
    return holding * np.maximum(x, 0) + backlog * np.maximum(-x, 0)

@jit(nopython=True)
def uniform_allocation(m, H):
    S = [1]
    step = H / (m + 1)
    for i in range(1, m + 1):
        S.append(int(np.round(1 + i * step)))
    S.append(H + 1)
    return S
       

In [ ]:
def compute_relative_profit_loss(rev_opt, rev_alg):
    rel_loss = [(rev_opt[i] - rev_alg[i]) / rev_opt[i] for i in range(len(rev_alg))]
    return rel_loss
    

 def solve_DP_estimate(m, D, dist, y_grid):
    '''
    Realization of Eq(1)
    
    Input: 
        D: D empirical distribution (for every re-pricing period, every p)
        dist: sample space of the m+1 noises distributions
    Output:
        rev_to_go: optimal revenue
        p_policy: optimal price (for every level y, every re-pricing period)
    '''
    D_new = np.array([np.array([D[k][i] for i in range(p_number)]) for k in range(1,m+2)])
    R = D_new * p_grid.reshape(1, -1)
    
    rev_to_go=np.empty((len(y_grid),m+1))
    p_policy=np.empty((len(y_grid),m+1))
    
    for k in range(m+1,0,-1):
        
        y_index = proj_idx(y_grid.reshape(-1,1,1)-D_new[k-1].reshape(1,-1,1)-dist[k-1].reshape(1,1,-1),y_grid)
        
        if k == m+1:
            # precompute proj(y-D[k][i]-noise_sample) and values of G()
            G_val = G(y_grid[y_index])
            temp = R[k-1].reshape(1,-1)-np.mean(G_val,axis=-1)
            temp_max = np.max(temp,axis=-1)
            
        else:
            # precompute proj(y-D[k][i]-noise_sample) and values of u_{k+1}()
            u_val = temp_max[y_index]
            temp = R[k-1].reshape(1,-1) + np.mean(u_val,axis=-1)
            temp_max = np.max(temp,axis=-1)
     
        rev_to_go[:,k-1] = temp_max
        p_policy[:,k-1]=p_grid[np.argmax(temp,axis=-1)]

    rev_to_go_df=pd.DataFrame(rev_to_go,index=y_grid,columns=range(1,m+2))
    p_policy_df=pd.DataFrame(p_policy,index=y_grid,columns=range(1,m+2))
    
    return rev_to_go_df, p_policy_df

    
# compute the revenue following a policy and a sequence of aggregated noise realization

def compute_rev(m, theta_S, y_ini, policy, noise_seq, y_grid):
    '''
    Compute Revenue (in an epoch)

    Input:
        theta_S: parameter to combine demand (basis eta)
        noise_seq: construct random demand part
        
        y_ini: order up to level
        policy: price (for every level y, every re-pricing period)
        y_grid: inventory grid
        
    Output:
        rev_epoch: revnue for this epoch
        p_used: the pricing we have used
        d_list: the demand we have used
    '''
    rev_epoch = 0
    p_used = []
    d_list = []
    y_level = y_ini # initialize the inventory level of the algorithm
    
    for k in range(1,m+2):
        p_k = policy.loc[y_level,k]
        p_used.append(p_k)
        d_k = np.dot(eta(p_k),theta_S[k-1])+noise_seq[k-1]
        d_list.append(d_k)

        y_level = proj(y_level-d_k,y_grid) # update the inventory level
        if k < m+1:
            rev_epoch = rev_epoch+p_k*d_k
        else:
            rev_epoch = rev_epoch+p_k*d_k-G(y_level)
        
    return rev_epoch, p_used, d_list
    

# compute the optimal revenue under the full-information setting
def solve_optimal_policy(m, sigma_S, theta_S, y_grid, monte_num=100): 
    '''
    Compute Optimal Policy (in an epoch)

    Input:
        m: pricing changing points
        sigma_S: used to generate random distribution of demand
        theta_S: parameter to combine demand basis eta
        noise_seq: construct random demand part
        y_grid: inventory grid
        
    Output:
        rev_epoch: revnue for this epoch
        p_used: the pricing we have used
        d_list: the demand we have used
    '''
    dist = [np.random.normal(0,sigma_S[k-1],monte_num) for k in range(1,m+2)]
    D = {k:[np.dot(theta_S[k-1],eta(p)) for p in p_grid] for k in range(1,m+2)}
    return solve_DP_estimate(m, D, dist, y_grid)
    
  

def compute_oracle(T, m, S, theta_S, noise_inst, y_ini_optimal,pricing_optimal, y_grid):
    '''
    learning algorithm with rare updates for a given instance of noises
    noise_inst is a list of T lists of m+1 random noises
    '''    
    rev_list_opt = []
    for t in range(1,T+1):
        # compute the revenue and prices of the optimal policy in epoch t
        rev_epoch_opt, price_opt_t, d_opt_t = compute_rev(m, theta_S, y_ini_optimal, pricing_optimal, noise_inst[t-1], y_grid)
        rev_list_opt.append(rev_epoch_opt)
        
    return rev_list_opt  

In [ ]:
def learning_algorithm(m, T, H, S, theta_S, noise_inst, y_ini_optimal, pricing_optimal, y_u, y_grid, p_u, d_l_shift=0, sample_max=500, MODIFIED=False, CONST_b=True, V_TYPE=None, NO_OPT=False):
    '''
    learning algorithm with rare updates for a given instance of noises
    noise_inst is a list of T lists of m+1 random noises
    '''
    # learning algorithm parameters
    eta_lip = max(np.linalg.norm(eta(p_grid[i])-eta(p_grid[j]))/abs(p_grid[i]-p_grid[j]) for i in range(p_number) for j in range(p_number) if i != j)
    L0 = max([np.sqrt(np.dot(theta_true[k],theta_true[k])) for k in range(H)])
    L = max(np.sqrt(np.dot(eta(p_u),eta(p_u))),holding,backlog,eta_lip,L0)
    T_2 = get_T_2(T)
    sigma_S = get_sigma_S(sigma, m, S)
    if CONST_b is False:
        V = get_V(T, V_TYPE)  
    # 1
    gamma = [np.sqrt(S[k]-S[k-1])*(L+sigma_m*np.sqrt(2*np.log((m+1)*(H*T*L)**2))) for k in range(1,m+2)]
    kapa = 2*L*max(gamma)*np.sqrt(2*dim*np.log(T*L**2+max(np.diff(S))))
    
    # lower bound of aggregated demand, used to compute the modified alg 2 (1)
    if MODIFIED == 1:
        d_l = np.array(get_d_l(p_u, theta_S, m)) - d_l_shift * np.array(sigma_S)
        d_l.tolist()
    else:
        d_l = get_d_l(p_u, theta_S, m)
    
    price_hist={} # store price decision of the algorithm
    d_hist={} # store the demand of the price decisions
    Lambda={} # store the design matrix
    theta={} # store estimated coefficients
    Delta_hist={} # store the UCB for past prices
    D={} # store estimated demand function
    rev_list=[] # store revenue for each epoch of the algorithm
    rev_list_opt=[] # store revenue for each epoch of the optimal offline policy
    det=[] # store the determinant of the design matrix
    y_alg=[] # store inventory decision of the algorithm
    # update_period=[]
    d_dtimesp=[] 
    
    for t in range(1,T+1):
        if CONST_b is False:
            # b = b(t), changing with epoch t
            b = [-(5+4*np.sin(V*np.pi*t/(2*T)))] * H
            theta_S = get_theta_S(a, b, m, S)
            
        sample={} # store the samples of noises to be used in DP in each epoch 
        if (CONST_b is False) and (NO_OPT is False):
            # compute the revenue and prices of the optimal policy in epoch t
            rev_to_go_optimal, pricing_optimal = solve_optimal_policy(m, sigma_S, theta_S, y_grid, 500)    
            # compute the optimal inventory level according  to u[1]
            y_ini_optimal = y_grid[np.asarray(rev_to_go_optimal[1]).argmax()]
            
        if t==1: # 2. initialization
            # update_period.append(t)
            y_ini = y_u #order the largest quantity
            y_alg.append(y_ini)
            price_hist[t] = np.zeros(m+1)+p_l # pricing the lowest for m+1 re-pricing time
            d_hist[t] = [np.dot(eta(price_hist[t][k-1]),theta_S[k-1]) + noise_inst[t-1][k-1] for k in range(1,m+2)]
            rev_list.append(np.dot(price_hist[t],d_hist[t])-G(proj(y_ini-sum(d_hist[t]),y_grid)))
            if NO_OPT is False:
                rev_epoch_opt, price_opt_t, d_opt_t = compute_rev(m, theta_S, y_ini_optimal, pricing_optimal, noise_inst[t-1], y_grid)
                # print(f'rev_epoch_opt={rev_epoch_opt}')
                rev_list_opt.append(rev_epoch_opt)
            
            theta[t] = [[0]*dim]*(m+1) # set theta_1 to be zero vector
            Lambda[t] = [(S[k]-S[k-1])*np.identity(dim) for k in range(1,m+2)] # set Lambda_1 to be identity matrix
            Delta_hist[t] = [gamma[k-1]*np.sqrt(np.dot(eta(price_hist[t][k-1]),inv(Lambda[t][k-1]) @ eta(price_hist[t][k-1]))) for k in range(1,m+2)]
            
            det = [np.linalg.det(Lambda[t][k-1]) for k in range(1,m+2)]
            
            # update Lambda[t+1] and (\sum d,\sum d*p)
            Lambda[t+1] = [Lambda[t][k-1]+np.outer(eta(price_hist[t][k-1]),eta(price_hist[t][k-1])) for k in range(1,m+2)] 
            
            d_dtimesp = [np.array([d_hist[t][k-1],d_hist[t][k-1]*price_hist[t][k-1]]) for k in range(1,m+2)]
            
        # for t in T_2 or exists k s.t. det(lambda_k) is doubled, estimate the parameters and compute UCB
        elif t in T_2 or sum(np.linalg.det(Lambda[t][k-1])>= 2*det[k-1] for k in range(1,m+2))>0: 
            # update_period.append(t)
            # print(f'start solving DP in period {t}')
            # update the detminant
            # 5.
            det=[np.linalg.det(Lambda[t][k-1]) for k in range(1,m+2)]
            # estimated coefficient
            theta[t]=[np.matmul(inv(Lambda[t][k-1]),d_dtimesp[k-1]) for k in range(1,m+2)]
            
            # for each k, compute Delta_tk(p) for p in grid, also for \bar D_t^k
            for k in range(1,m+2):
                Delta = [gamma[k-1]*np.sqrt(np.dot(eta(p_grid[i]),inv(Lambda[t][k-1]) @ eta(p_grid[i]))) for i in range(p_number)]
                UCB = [np.dot(theta[t][k-1],eta(p_grid[i])) for i in range(p_number)] # eta(p) times theta_t^k
                # 6. different bar{D_t^k}(p)
                if MODIFIED is False:
                    D[k] = [min(UCB[i]+Delta[i], np.dot(theta_S[k-1],eta(p_l)), np.dot(theta_S[k-1],eta(p_u))+(S[k]-S[k-1])*(L**2)*(p_u-p_grid[i])) for i in range(p_number)]
                elif MODIFIED == 1:
                    D[k] = [min(UCB[i]+Delta[i], d_l[k-1]+(S[k]-S[k-1])*(L**2)*(p_u-p_grid[i])) for i in range(p_number)]
                elif MODIFIED == 2:
                    D[k] = [UCB[i]+Delta[i] for i in range(p_number)]

                H_index = [s for s in range(1,t) if np.sqrt(t)*Delta_hist[s][k-1]<kapa]
                # random choose sample_max number of histories
                # Update: choose last sample_max number of histories 
                if len(H_index) > sample_max:
                    # H_small=np.random.choice(H_index, sample_max, replace=False)
                    H_small = H_index[-sample_max:]
                else:
                    H_small = H_index
                # 7. hat{\mu}_t^k 
                sample[k-1] = np.array([d_hist[s][k-1]-np.dot(theta[s][k-1],eta(price_hist[s][k-1])) for s in H_small])      
            
            # 8. solve the optimal inventory level for the estimated parameters and distributions
            rev_to_go_estimate, p_policy_alg = solve_DP_estimate(m, D,sample,y_grid)
            # 10.
            y_ini = y_grid[np.asarray(rev_to_go_estimate[1]).argmax()]
            y_alg.append(y_ini)
            
            # 9. compute the revenue and prices in epoch t
            rev_epoch, price_hist[t],d_hist[t] = compute_rev(m, theta_S, y_ini,p_policy_alg,noise_inst[t-1], y_grid)
            rev_list.append(rev_epoch)
            # compute the revenue and prices of the optimal policy in epoch t
            if NO_OPT is False:
                rev_epoch_opt, price_opt_t, d_opt_t = compute_rev(m, theta_S, y_ini_optimal,pricing_optimal,noise_inst[t-1], y_grid)
                # print(f'rev_epoch_opt={rev_epoch_opt}')
                rev_list_opt.append(rev_epoch_opt)
            
            # update Lambda[t+1]
            Lambda[t+1]=[Lambda[t][k-1]+np.outer(eta(price_hist[t][k-1]),eta(price_hist[t][k-1])) for k in range(1,m+2)] 
            d_dtimesp=[d_dtimesp[k-1]+np.array([d_hist[t][k-1],d_hist[t][k-1]*price_hist[t][k-1]]) for k in range(1,m+2)]
            
            # store the Delta of this epoch
            Delta_hist[t]=[gamma[k-1]*np.sqrt(np.dot(eta(price_hist[t][k-1]),inv(Lambda[t][k-1]) @ eta(price_hist[t][k-1]))) for k in range(1,m+2)]
            
        else:
            theta[t]=theta[t-1]
            
            # compute the revenue and prices in epoch t
            rev_epoch,price_hist[t],d_hist[t]=compute_rev(m, theta_S, y_ini,p_policy_alg,noise_inst[t-1], y_grid)
            # print(f'rev_epoch={rev_epoch}')
            rev_list.append(rev_epoch)
            y_alg.append(y_ini)

            if NO_OPT is False:
            # compute the revenue and prices of the optimal policy in epoch t
                rev_epoch_opt,price_opt_t,d_opt_t=compute_rev(m, theta_S, y_ini_optimal,pricing_optimal,noise_inst[t-1], y_grid)
                rev_list_opt.append(rev_epoch_opt)
            
            # update Lambda[t+1]
            Lambda[t+1]=[Lambda[t][k-1]+np.outer(eta(price_hist[t][k-1]),eta(price_hist[t][k-1])) for k in range(1,m+2)] 
            d_dtimesp=[d_dtimesp[k-1]+np.array([d_hist[t][k-1],d_hist[t][k-1]*price_hist[t][k-1]]) for k in range(1,m+2)]
            
            # store the Delta of this epoch
            Delta_hist[t]=[gamma[k-1]*np.sqrt(np.dot(eta(price_hist[t][k-1]),inv(Lambda[t][k-1]) @ eta(price_hist[t][k-1]))) for k in range(1,m+2)]

    return rev_list, rev_list_opt, y_alg, theta    

In [ ]:
def Exp_Exp_algorithm(m, T, H, S, theta_S, noise_inst, y_grid, y_ini_optimal, pricing_optimal, d_u, sigma_S, sample_max=500, FRAC=1, CONST_b=True, V_TYPE=None, NO_OPT=False):    
    '''
    Step 1
    Explore good policy
    '''
    # learning algorithm parameters
    eta_lip = max(np.linalg.norm(eta(p_grid[i])-eta(p_grid[j]))/abs(p_grid[i]-p_grid[j]) for i in range(p_number) for j in range(p_number) if i != j)
    L0 = max([np.sqrt(np.dot(theta_true[k],theta_true[k])) for k in range(H)])
    L = max(np.sqrt(np.dot(eta(p_u),eta(p_u))), holding, backlog, eta_lip, L0)

    if CONST_b is False:
        V = get_V(T, V_TYPE)
        sigma_S = get_sigma_S(sigma, m, S)
            
    # 1
    gamma = [np.sqrt(S[k]-S[k-1])*(L+sigma_m*np.sqrt(2*np.log((m+1)*(H*T*L)**2))) for k in range(1,m+2)]
    kapa = 2*L*max(gamma)*np.sqrt(2*dim*np.log(T*L**2+max(np.diff(S))))
    
    # upper bound of y level according to upper bound of aggregated demand
    d_u = [d_u[i] + 3 * sigma_S[i] for i in range(len(sigma_S))]
    y_0 = sum(d_u)
    
    price_hist={} # store price decision of the algorithm
    d_hist={} # store the demand of the price decisions
    Lambda={} # store the design matrix
    theta={} # store estimated coefficients
    Delta_hist={} # store the UCB for past prices
    D={} # store estimated demand function
    rev_list=[] # store revenue for each epoch of the algorithm
    rev_list_opt=[] # store revenue for each epoch of the optimal offline policy
    det=[] # store the determinant of the design matrix
    # update_period=[]
    d_dtimesp=[] 
    if FRAC == 1:
        T0 = int(np.sqrt(T))
    elif FRAC == 2:
        T0 = int(pow(T,2/3))
    else:
        print('FRAC = 1 or 2. Now this function support 1/2 and 2/3.')
        
    # 2
    #------------t = 1:T0------------#    
    for t in range(1,T0+1):
        if CONST_b is False:
            # b = b(t), changing with epoch t
            b = [-(5+4*np.sin(V*np.pi*t/(2*T)))] * H
            theta_S = get_theta_S(a, b, m, S)
            
        sample={} 
        # 3
        y_ini = y_0
        # 4 5
        price_hist[t] = np.random.sample(m+1) * (p_u - p_l) +  p_l
        d_hist[t] = [np.dot(eta(price_hist[t][k-1]),theta_S[k-1]) + noise_inst[t-1][k-1] for k in range(1,m+2)]
        
        if t==1:
            theta[t] = [[0]*dim] * (m+1) # set theta_1 to be zero vector
            Lambda[t] = [[S[k]-S[k-1]] * np.identity(dim) for k in range(1,m+2)] # set Lambda_1 to be identity matrix
            Delta_hist[t] = [gamma[k-1] * np.sqrt(np.dot(eta(price_hist[t][k-1]),inv(Lambda[t][k-1]) @ eta(price_hist[t][k-1]))) for k in range(1, m+2)]
            det = [np.linalg.det(Lambda[t][k-1]) for k in range(1,m+2)]
            # update Lambda[t+1] and (\sum d,\sum d*p)
            Lambda[t+1] = [Lambda[t][k-1]+np.outer(eta(price_hist[t][k-1]),eta(price_hist[t][k-1])) for k in range(1,m+2)] 
            d_dtimesp = [np.array([d_hist[t][k-1],d_hist[t][k-1]*price_hist[t][k-1]]) for k in range(1, m+2)]
            
        else:
            # update the detminant
            det = [np.linalg.det(Lambda[t][k-1]) for k in range(1,m+2)]
            theta[t] = [np.matmul(inv(Lambda[t][k-1]),d_dtimesp[k-1]) for k in range(1,m+2)]
            
            # for each k, compute Delta_tk(p) for p in grid, also for \bar D_t^k
            for k in range(1, m+2):
                Delta = [gamma[k-1]*np.sqrt(np.dot(eta(p_grid[i]),inv(Lambda[t][k-1]) @ eta(p_grid[i]))) for i in range(p_number)]
                UCB = [np.dot(theta[t][k-1],eta(p_grid[i])) for i in range(p_number)] # eta(p) times theta_t^k
                D[k] = [min(UCB[i] + Delta[i], np.dot(theta_S[k-1], eta(p_l)), np.dot(theta_S[k-1],eta(p_u)) + 1*(L**2)*(p_u-p_grid[i])) for i in range(p_number)]
                H_index = [s for s in range(1,t) if np.sqrt(t) * Delta_hist[s][k-1] < kapa]

                if len(H_index) > sample_max:
                    H_small = H_index[-sample_max:]
                else:
                    H_small = H_index
                sample[k-1] = np.array([d_hist[s][k-1] - np.dot(theta[s][k-1],eta(price_hist[s][k-1])) for s in H_small])
            
            # update Lambda[t+1]
            Lambda[t+1] = [Lambda[t][k-1] +np.outer(eta(price_hist[t][k-1]),eta(price_hist[t][k-1])) for k in range(1,m+2)] 
            d_dtimesp=[d_dtimesp[k-1]+np.array([d_hist[t][k-1],d_hist[t][k-1]*price_hist[t][k-1]]) for k in range(1,m+2)]
            
            # store the Delta of this epoch
            Delta_hist[t]=[gamma[k-1]*np.sqrt(np.dot(eta(price_hist[t][k-1]),inv(Lambda[t][k-1]) @ eta(price_hist[t][k-1]))) for k in range(1,m+2)]

        # compute the revenue
        rev_list.append(np.dot(price_hist[t],d_hist[t])-G(proj(y_ini-sum(d_hist[t]), y_grid)))
        if NO_OPT is False:
            # compute the revenue and prices of the optimal policy in epoch t
            rev_epoch_opt, price_opt_t, d_opt_t = compute_rev(m, theta_S, y_ini_optimal, pricing_optimal, noise_inst[t-1], y_grid)
            rev_list_opt.append(rev_epoch_opt)

    #------------t = T0+1------------#
    theta[t]=[np.matmul(inv(Lambda[t][k-1]),d_dtimesp[k-1]) for k in range(1,m+2)]
    
    for k in range(1,m+2):
        Delta = [gamma[k-1] * np.sqrt(np.dot(eta(p_grid[i]),inv(Lambda[t][k-1]) @ eta(p_grid[i]))) for i in range(p_number)]
        UCB = [np.dot(theta[t][k-1],eta(p_grid[i])) for i in range(p_number)] # eta(p) times theta_t^k
        # 6. estimate: D_t^k(p)
        D[k] = [min(UCB[i]+Delta[i],np.dot(theta_S[k-1],eta(p_l)),np.dot(theta_S[k-1],eta(p_u))+(S[k]-S[k-1])*(L**2)*(p_u-p_grid[i])) for i in range(p_number)]
        H_index = [s for s in range(1,t) if np.sqrt(t)*Delta_hist[s][k-1]<kapa]
        if len(H_index) > sample_max:
            H_small = H_index[-sample_max:]
        else:
            H_small = H_index
        # 7. estimate hat{\mu}_t^k 
        sample[k-1] = np.array([d_hist[s][k-1]-np.dot(theta[s][k-1],eta(price_hist[s][k-1])) for s in H_small])    

    # 8. 9. solve DP using the estimated parameters and distributions
    rev_to_go_estimate, p_policy_alg = solve_DP_estimate(m, D, sample, y_grid)
    y_new = y_grid[np.asarray(rev_to_go_estimate[1]).argmax()]
    
    '''
    Step 2
    Exploit the policy has been learnt
    '''

    # 10.
    #------------t = T0+1:T------------#
    for t in range(T0+1,T+1):
        if CONST_b is False:
            # b = b(t), changing with epoch t
            b = [-(5+4*np.sin(V*np.pi*t/(2*T)))] * H
            theta_S = get_theta_S(a, b, m, S)
        theta[t]=theta[t-1]
        # compute the revenue and prices in epoch t
        rev_epoch, price_hist[t], d_hist[t] = compute_rev(m, theta_S, y_new, p_policy_alg, noise_inst[t-1], y_grid)
        rev_list.append(rev_epoch)
        if NO_OPT is False:
            # compute the revenue and prices of the optimal policy in epoch t
            rev_epoch_opt, price_opt_t,d_opt_t = compute_rev(m, theta_S, y_ini_optimal, pricing_optimal, noise_inst[t-1], y_grid)
            rev_list_opt.append(rev_epoch_opt)

    return rev_list, rev_list_opt, y_new, theta    

In [ ]:
def learning_regret(m, M, T, H, p_u, d_l_shift=0, sample_max=500, UNI_ALOC=False, ALG=2, FRAC=1, REGRET=True, MODIFIED=False, LOG_T=False, CONST_b = True, V_TYPE=None, TOTAL_REGRET=False, NO_OPT=False):
    """
    Calculate the learning regret or profit in a pricing algorithm.

    Input:
        m (int): Number of pricing change points.
        M (int): Repetition times.
        T (int): Number of epochs.
        H (int): Number of periods.
        p_u (float): Upper price limit.
        d_l_shift (int, optional): For Modified Alg 2(1), d_l <- d_l - d_l_shift * sigma.
        sample_max (int, optional): Number of latest epochs for computing DP. Defaults to 500.
        UNI_ALOC (bool, optional): Use uniform allocation if True, otherwise optimal allocation. Defaults to False.
        ALG (int, optional): Algorithm choice, either Alg2 (2) or Exp-Exp (3). Defaults to 2.
        FRAC (int, optional): Fraction choice for Exp-Exp Alg, either T_0 = T^(1/2) (1) or T_0 = T^(2/3) (2). Defaults to 1.
        REGRET (bool, optional): Compute regret if True, otherwise compute profit. Defaults to True.
        MODIFIED (bool, optional): Use modified Alg2 if True, otherwise use Alg2. Defaults to False.
        LOG_T (bool, optional): Use log_2(T) if True, otherwise use False. Defaults to False.
        CONST_b (bool, optional): Use True if b is const with epoch t, otherwise use False. Defaults to True.
        V_TYPE (str, optional): Use non-const b. 'LOG', '0.2' or '0.5'. Defaults to None.
        TOTAL_REGRET (bool, optional): Compute total regret if True, otherwise computing learning regret. Defaults to False.
        NO_OPT (bool, optional): Compute OPT revenue if False, otherwise True. Defaults to False.

    Output:
        float: Average regret if REGRET is True, otherwise average profit.
    """
    
    noise_all={}
    for rep_num in range(M):
        # generate noise instances
        noise_seq=[[np.random.normal(0,sigma[i]) for i in range(H)] for t in range(T)]
        noise_all[rep_num]=noise_seq

    # print(f'--------------T = {T}--------------')
    # print(f'--------------m = {m}--------------')
    # optimal allocation
    S = get_S(m, H, UNI_ALOC)
    
    # aggregated parameters
    theta_S = get_theta_S(a, b, m, S)
    sigma_S = get_sigma_S(sigma, m, S)
    T_2 = get_T_2(T)
    # T_2 = np.arange(1000,4500,500)

    # upper bound and lower bound of aggregated demand
    d_l = get_d_l(p_u, theta_S, m)
    d_u = get_d_l(p_l, theta_S, m)
    # define the space of the inventory level, P(X>4 sigma_m)<= e^{-8}=0.00033...
    y_u = get_y_l(d_u, H, sigma_m)
    y_l = -y_u  
    # y_grid=np.linspace(y_l,y_u,100)
    y_grid = np.arange(y_l,y_u+1) # only consider integer inventory
    # aggregated noise
    noise_S_all = {}
    for rep_num in range(M):
        noise_S_all[rep_num] = get_noise_S(noise_all[rep_num], m, T, S)

    if NO_OPT is False:
        # compute the revenue to go function and the pricing policy of the optimal policy
        rev_to_go_optimal, pricing_optimal = solve_optimal_policy(m, sigma_S, theta_S, y_grid, 500)    
        # compute the optimal inventory level according  to u[1]
        y_ini_optimal = y_grid[np.asarray(rev_to_go_optimal[1]).argmax()]
    else:
        y_ini_optimal = None
        pricing_optimal = None
    
        # print('Clairvoyant optimal inventory level:', y_ini_optimal)
        # print('Clairvoyant optimal revenue per epoch:', max(rev_to_go_optimal[1]))

    if TOTAL_REGRET:
        # define new parameters when m = H-1
        noise_H_all = {}
        S_H = get_S(H-1, H, UNI_ALOC)
        theta_H = get_theta_S(a, b, H-1, S_H)
        sigma_H = get_sigma_S(sigma, H-1, S_H)
        rev_opt_H = [[[0] for col in range(T)] for row in range(M)]
        d_l_h = get_d_l(p_u, theta_H, H-1)
        d_u_h = get_d_l(p_l, theta_H, H-1)
        y_u_h = get_y_l(d_u_h, H, sigma_m)
        y_l_h = - y_u_h
        y_grid_h = np.arange(y_l_h, y_u_h + 1)

        # solve DP when m = H-1
        rev_to_go_opt_H, pricing_opt_H = solve_optimal_policy(H-1, sigma_H, theta_H, y_grid_h, 500)
        y_ini_opt_H = y_grid_h[np.asarray(rev_to_go_opt_H[1]).argmax()]
            
        for rep_num in range(M):
            rev_opt_H[rep_num] = []
            noise_H_all[rep_num] = get_noise_S(noise_all[rep_num], H-1, T, S_H)
            rev_opt_H[rep_num] = compute_oracle(T, H-1, S, theta_H, noise_H_all[rep_num], y_ini_opt_H, pricing_opt_H, y_grid_h)
        if LOG_T:
            rev_H = [np.mean(mean_row(rev_opt_H)[0:t])  for t in T_2]
            rev_H_sum = [np.sum(mean_row(rev_opt_H)[0:t])  for t in T_2]
        else:
            rev_H = np.mean(mean_row(rev_opt_H))
            rev_H_sum = np.sum(mean_row(rev_opt_H))
    
    output={}
    rev_alg = [[[0] for col in range(T)] for row in range(M)]
    rev_opt = [[[0] for col in range(T)] for row in range(M)]
    
    for rep_num in range(M):
        # print('------------------------------------------------------')
        # print('start ', rep_num+1, ' round')
        if ALG == 2:
            output[rep_num] = learning_algorithm(m, T, H, S, theta_S, noise_S_all[rep_num], y_ini_optimal, pricing_optimal, y_u, y_grid, p_u, d_l_shift=d_l_shift, sample_max=sample_max, MODIFIED=MODIFIED, CONST_b=CONST_b, V_TYPE=V_TYPE, NO_OPT=NO_OPT)
        elif ALG == 3:
            output[rep_num] = Exp_Exp_algorithm(m, T, H, S, theta_S, noise_S_all[rep_num], y_grid, y_ini_optimal, pricing_optimal, d_u=d_u, sigma_S=sigma_S, sample_max=500, FRAC=FRAC, CONST_b=CONST_b, V_TYPE=V_TYPE, NO_OPT=NO_OPT)
        else:
            print('ALG = 2 or 3. Now this function supports the computation of Alg 2 and 3.')
        
        rev_alg[rep_num] = output[rep_num][0]
        if NO_OPT is False:
            rev_opt[rep_num] = output[rep_num][1]

    if LOG_T:
        rev_alg_sum = [np.sum(mean_row(rev_alg)[0:t])  for t in T_2]
        rev_alg = [np.mean(mean_row(rev_alg)[0:t])  for t in T_2]
        if NO_OPT is False:
            rev_opt_sum = [np.sum(mean_row(rev_opt)[0:t])  for t in T_2]
            rev_opt = [np.mean(mean_row(rev_opt)[0:t])  for t in T_2]
    else:
        rev_alg_sum = np.sum(mean_row(rev_alg))
        rev_alg = np.mean(mean_row(rev_alg))
        if NO_OPT is False:
            rev_opt_sum = np.sum(mean_row(rev_opt))
            rev_opt = np.mean(mean_row(rev_opt))
    
    if REGRET:
        regret = minus(rev_opt_sum, rev_alg_sum)
        if TOTAL_REGRET:
            regret2 = minus(rev_H_sum, rev_alg_sum)
            return regret, regret2
        return regret
    if NO_OPT is False:
        return rev_alg, rev_opt
    return rev_alg
    

In [ ]:
# def get_first_regret(m, M, T, H, p_u, d_l_shift=0, sample_max=500, UNI_ALOC=False, ALG=2, FRAC=1, REGRET=True, MODIFIED=False, LOG_T=False, CONST_b = True, V_TYPE=None, TOTAL_REGRET=False):
#     # m is x axis, solve 2 DP: m = H-1; and m; minus
#     #--------------------------------------------
#     # define new parameters when m = H-1
#     def get_rev_opt(m):
#         noise_H_all = {}
#         S_H = get_S(m, H, UNI_ALOC)
#         theta_H = get_theta_S(a, b, m, S_H)
#         sigma_H = get_sigma_S(sigma, m, S_H)
#         rev_opt_H = [[[0] for col in range(T)] for row in range(M)]
#         d_l_h = get_d_l(p_u, theta_H, m)
#         d_u_h = get_d_l(p_l, theta_H, m)
#         y_u_h = get_y_l(d_u_h, H, sigma_m)
#         y_l_h = - y_u_h
#         y_grid_h = np.arange(y_l_h, y_u_h + 1)
        
#         # solve DP when m = H-1
#         rev_to_go_opt_H, pricing_opt_H = solve_optimal_policy(m, sigma_H, theta_H, y_grid_h, 500)
#         y_ini_opt_H = y_grid_h[np.asarray(rev_to_go_opt_H[1]).argmax()]
            
#         for rep_num in range(M):
#             rev_opt_H[rep_num] = []
#             noise_H_all[rep_num] = get_noise_S(noise_all[rep_num], m, T, S_H)
#             rev_opt_H[rep_num] = compute_oracle(T, m, S, theta_H, noise_H_all[rep_num], y_ini_opt_H, pricing_opt_H, y_grid_h)
    
#         rev_H = np.mean(mean_row(rev_opt_H))
#         rev_H_sum = np.sum(mean_row(rev_opt_H))
#         return rev_H, rev_H_sum

#     rev_H, rev_H_sum = get_rev_opt(H-1)
#     rev_m, rev_m_sum = get_rev_opt(m)
    
#     return rev_H - rev_m
    

# Parameters 

In [ ]:
# define the space of prices
p_u=1
p_l=0
p_number=10
p_grid=np.linspace(p_l,p_u,p_number)
dim=len(eta(p_u))

# holding and backlogging costs
holding=0.05
backlog=1
H = 5
a=[30]*H
b=[-20]*H

# a=[10]*H
# b=[-5]*H

# a = [20,30] * (H//2)
# b = [-5,-20] * (H//2)

# a = [20,20] * (H//2)
# b = [-5, -5] * (H//2)
# a.append(20)
# b.append(-5)

# a=range(20,20+H)
# b=[-5]*H

# a = [24,23,22,21,20]
# a = [20, 21, 22, 21, 20]
# a = [20, 25, 20, 25, 20]

theta_true=[x for x in zip(a,b)] # true demand-price function is D(p) = 20-5p

# noise parameter, normal
sigma=[2]*H
sigma_m=max(sigma)

T = 2**12
m = 3
M = 10 # repeat M times of the algorithm

# Experiment

## Compare Algorithm 2 with the modified Algorithm 2

In [ ]:
# # compute
# # compute Exp-exp
# rev_alg1, rev_opt1 = learning_regret(m, M, T, H, p_u, REGRET=False, ALG=3, LOG_T=True)

# # compute Alg2
# rev_alg2, rev_opt2 = learning_regret(m, M, T, H, p_u, REGRET=False, ALG=2, LOG_T=True)

# # compute Alg2 (1)
# rev_alg3, rev_opt3 = learning_regret(m, M, T, H, p_u, REGRET=False, ALG=2, MODIFIED=1, LOG_T=True)
# # rev_alg3_, rev_opt3_ = learning_regret(m, M, T, H, p_u, REGRET=False, ALG=2, MODIFIED=1, LOG_T=True, d_l_shift=1)
# # rev_alg3__, rev_opt3__ = learning_regret(m, M, T, H, p_u, REGRET=False, ALG=2, MODIFIED=1, LOG_T=True, d_l_shift=2)

# # compute Alg2 (2)
# rev_alg4, rev_opt4 = learning_regret(m, M, T, H, p_u, REGRET=False, ALG=2, MODIFIED=2, LOG_T=True)

# # store
# with open('results/output_1_a.pkl', 'wb') as f:
#     pickle.dump([rev_alg1, rev_opt1, rev_alg2, rev_opt2, rev_alg3, rev_opt3, rev_alg4, rev_opt4], f)
# f.close()

# # plot
# T_2 = get_T_2(T)
# plt.plot(range(1,len(T_2)+1),rev_alg1,label='Exp-Exp', marker='.')
# plt.plot(range(1,len(T_2)+1),rev_alg3,label='Modified Alg2 (1)', marker='.')
# plt.plot(range(1,len(T_2)+1),rev_opt1,label='Oracle solution', linestyle='--') 
# plt.xlabel(r'$log_2(T)$')
# plt.ylabel('avg profit')
# plt.title('Average Profit For Modified Alg2 (1)')
# plt.legend()

# T_2 = get_T_2(T)
# plt.plot(range(1,len(T_2)+1),rev_alg1,label='Exp-Exp', marker='.')
# plt.plot(range(1,len(T_2)+1),rev_alg4,label='Modified Alg2 (2)', marker='.')
# plt.plot(range(1,len(T_2)+1),rev_opt1,label='Oracle solution', linestyle='--') 
# plt.xlabel(r'$log_2(T)$')
# plt.ylabel('avg profit')
# plt.title('Average Profit For Modified Alg2 (2)')
# plt.legend()

# T_2 = get_T_2(T)
# plt.plot(range(1,len(T_2)+1),rev_alg2,label='Algorithm 2', marker='.')
# plt.plot(range(1,len(T_2)+1),rev_opt1,label='Oracle solution', linestyle='--') 
# plt.plot(range(1,len(T_2)+1),rev_alg3,label='1st alternative', marker='.')
# plt.plot(range(1,len(T_2)+1),rev_alg4,label='2nd alternative', marker='.')
# plt.xlabel(r'$log_2(T)$')
# plt.ylabel('avg profit')
# plt.title('Average profit for different UCB constructions')
# plt.legend()

# # Average profit for different UCB constructions\\
# # legend: Oracle solution, Algorithm 2, 1st alternative, 2nd alternative\\

# T_2 = get_T_2(T)
# plt.plot(range(1,len(T_2)+1),rev_opt1,label='Oracle solution', linestyle='--') 
# plt.plot(range(1,len(T_2)+1),rev_alg2,label='Algorithm 2 with cap', marker='.')
# plt.plot(range(1,len(T_2)+1),rev_alg3,label='Algorithm 2', marker='.')
# plt.plot(range(1,len(T_2)+1),rev_alg4,label='2nd alternative', marker='.')
# plt.xlabel(r'$log_2(T)$')
# plt.ylabel('avg profit')
# plt.title('Average profit for different UCB constructions')
# plt.legend()

# # reload 
# with open('results/output_1_a.pkl', 'rb') as f:
#     rev_alg1, rev_opt1, rev_alg2, rev_opt2, rev_alg3, rev_opt3, rev_alg4, rev_opt4 = pickle.load(f)
# f.close()

In [ ]:
# # compute old
# p_u_range = np.arange(1,4.1,0.2)
# output_m1 = []
# output_m2 = []
# for i, p_u in enumerate(p_u_range):
#         print(f'--------------- p_u = {p_u}----------------')
#         rev_alg, rev_opt = learning_regret(m, M, T, H, p_u, REGRET=False, MODIFIED=1)
#         output_m1.append(rev_alg)
#         output_m2.append(rev_opt)   

# output_m1_ = []
# output_m2_ = []
# for i, p_u in enumerate(p_u_range):
#         print(f'--------------- p_u = {p_u}----------------')
#         rev_alg, rev_opt = learning_regret(m, M, T, H, p_u, REGRET=False, MODIFIED=2)
#         output_m1_.append(rev_alg)
#         output_m2_.append(rev_opt)   

In [ ]:
# # compute new

# output_m1 = []
# output_m2 = []

# output_m1_305 = []
# output_m2_305 = []

# d_l_shift_range = np.arange(0,2.2,0.2)
# for i, d_l_shift in enumerate(d_l_shift_range):
#         print(f'--------------- d_l_shift = {d_l_shift}----------------')
#         rev_alg, rev_opt = learning_regret(m, M, T, H, p_u, d_l_shift=d_l_shift, REGRET=False, MODIFIED=1)
#         output_m1_305.append(rev_alg)
#         output_m2_305.append(rev_opt)   

# output_m1_3020 = []
# output_m2_3020 = []
# d_l_shift_range = np.arange(0,2.2,0.2)
# for i, d_l_shift in enumerate(d_l_shift_range):
#         print(f'--------------- d_l_shift = {d_l_shift}----------------')
#         rev_alg, rev_opt = learning_regret(m, M, T, H, p_u, d_l_shift=d_l_shift, REGRET=False, MODIFIED=1)
#         output_m1_3020.append(rev_alg)
#         output_m2_3020.append(rev_opt)   

# output_m1_ = []
# output_m2_ = []

# d_l_shift_range = np.arange(0,1.2,0.2)
# for i, d_l_shift in enumerate(d_l_shift_range):
#         print(f'--------------- d_l_shift = {d_l_shift}----------------')
#         rev_alg, rev_opt = learning_regret(m, M, T, H, p_u, d_l_shift=d_l_shift, REGRET=False, MODIFIED=2)
#         output_m1_.append(rev_alg)
#         output_m2_.append(rev_opt)    

# # plot
# # d_l_range = np.min([np.array(get_d_l(p_u, get_theta_S(a, b, m, get_S(m, H, False)), m)) - d_l_shift * sigma_m  for d_l_shift in d_l_shift_range], axis=1) 
# d_l_range = np.arange(0,2.1,0.2)
# plt.plot(d_l_range, output_m1_305, label='1st alternative',marker='.')
# plt.plot(d_l_range, output_m2_305, label='Oracle solution', linestyle='--')
# plt.xlabel('multiplier')
# plt.ylabel('avg profit')
# plt.title(r'Average profit for different deviations ($a_i=30, b_i=-5$)')
# plt.legend()

# # plot
# # d_l_range = np.min([np.array(get_d_l(p_u, get_theta_S(a, b, m, get_S(m, H, False)), m)) - d_l_shift * sigma_m  for d_l_shift in d_l_shift_range], axis=1) 
# d_l_range = np.arange(0,2.1,0.2)
# plt.plot(d_l_range, output_m1_3020, label='1st alternative',marker='.')
# plt.plot(d_l_range, output_m2_3020, label='Oracle solution', linestyle='--')
# plt.xlabel('multiplier')
# plt.ylabel('avg profit')
# # plt.text(x = 1.5, y = 146, s = r'$T = 2^{10}, H ... 10, m=5, a = 20, b = -5$')
# plt.title(r'Average profit for different deviations ($a_i=30, b_i=-20$)')
# plt.ylim(45,56)
# plt.legend()

# # store
# with open('results/output_1_b_new_.pkl', 'wb') as f:
#     pickle.dump([output_m1_305, output_m2_305, output_m1_3020, output_m2_3020], f)
# f.close()

# # store
# with open('results/output_1_b_new.pkl', 'wb') as f:
#     pickle.dump([output_m1, output_m2, output_m1_, output_m2_], f)
# f.close()

# reload
with open('results/output_1_b_new.pkl', 'rb') as f:
    output_m1, output_m2, output_m1_, output_m2_ = pickle.load(f)
f.close()

## Nonstationary demand functions over epochs

In [ ]:
# # compute
# # LOG
# rev_alg5 = learning_regret(m, M, T, H, p_u, REGRET=False, ALG=2, LOG_T=True, CONST_b=False, V_TYPE='LOG', NO_OPT=True)

# rev_alg5_1 = learning_regret(m, M, T, H, p_u, REGRET=False, ALG=3, LOG_T=True, CONST_b=False, V_TYPE='LOG', NO_OPT=True)
# rev_alg5_2 = learning_regret(m, M, T, H, p_u, REGRET=False, ALG=3, LOG_T=True, CONST_b=False, V_TYPE='LOG', FRAC=2, NO_OPT=True)

# # T^0.2 
# rev_alg6 = learning_regret(m, M, T, H, p_u, REGRET=False, ALG=2, LOG_T=True, CONST_b=False, V_TYPE='0.2', NO_OPT=True )

# rev_alg6_1 = learning_regret(m, M, T, H, p_u, REGRET=False, ALG=3, LOG_T=True, CONST_b=False, V_TYPE='0.2', NO_OPT=True )
# rev_alg6_2 = learning_regret(m, M, T, H, p_u, REGRET=False, ALG=3, LOG_T=True, CONST_b=False, V_TYPE='0.2', FRAC=2, NO_OPT=True)

# # T_0.5
# rev_alg7 = learning_regret(m, M, T, H, p_u, REGRET=False, ALG=2, LOG_T=True, CONST_b=False, V_TYPE='0.5', NO_OPT=True )

# rev_alg7_1 = learning_regret(m, M, T, H, p_u, REGRET=False, ALG=3, LOG_T=True, CONST_b=False, V_TYPE='0.5', NO_OPT=True )
# rev_alg7_2 = learning_regret(m, M, T, H, p_u, REGRET=False, ALG=3, LOG_T=True, CONST_b=False, V_TYPE='0.5', FRAC=2, NO_OPT=True)

# with open('results/output_2_TRUE_.pkl', 'rb') as f:
#     _, rev_opt5, _, rev_opt6, _, rev_opt7 = pickle.load(f)
# f.close()

# # compute LOG changing b
# rev_alg5, rev_opt5 = learning_regret(m, 1, T, H, p_u, REGRET=False, ALG=2, LOG_T=True, CONST_b=False, V_TYPE='LOG' )
# rev_alg5_1, rev_opt5_1 = learning_regret(m, M, T, H, p_u, REGRET=False, ALG=3, LOG_T=True, CONST_b=False, V_TYPE='LOG' )
# rev_alg5_2, rev_opt5_2 = learning_regret(m, M, T, H, p_u, REGRET=False, ALG=3, LOG_T=True, CONST_b=False, V_TYPE='LOG', FRAC=2)

# # compute T^0.2 changing b
# rev_alg6, rev_opt6 = learning_regret(m, M, T, H, p_u, REGRET=False, ALG=2, MODIFIED=2, LOG_T=True, CONST_b=False, V_TYPE='0.2' )
# rev_alg6_1, rev_opt6_1 = learning_regret(m, M, T, H, p_u, REGRET=False, ALG=3, LOG_T=True, CONST_b=False, V_TYPE='0.2' )
# rev_alg6_2, rev_opt6_2 = learning_regret(m, M, T, H, p_u, REGRET=False, ALG=3, LOG_T=True, CONST_b=False, V_TYPE='0.2', FRAC=2)

# # compute T_0.5 changing b
# rev_alg7, rev_opt7 = learning_regret(m, M, T, H, p_u, REGRET=False, ALG=2, MODIFIED=2, LOG_T=True, CONST_b=False, V_TYPE='0.5' )
# rev_alg7_1, rev_opt7_1 = learning_regret(m, M, T, H, p_u, REGRET=False, ALG=3, LOG_T=True, CONST_b=False, V_TYPE='0.5' )
# rev_alg7_2, rev_opt7_2 = learning_regret(m, M, T, H, p_u, REGRET=False, ALG=3, LOG_T=True, CONST_b=False, V_TYPE='0.5', FRAC=2)

# # plot
# T_2 = get_T_2(T)
# plt.plot(range(1,len(T_2)+1),rev_alg5_tmp,label=r'Algorithm 2', marker='.')
# plt.plot(range(1,len(T_2)+1),rev_opt5,label='Oracle solution', linestyle='--')
# # plt.plot(range(1,len(T_2)+1),rev_alg5_1,label=r'$T_0=T^{\frac{1}{2}}$', marker='.')
# # plt.plot(range(1,len(T_2)+1),rev_alg5_2,label=r'$T_0=T^{\frac{2}{3}}$', marker='.')
# plt.xlabel(r'$log_2(T)$')
# plt.ylabel('avg profit')
# plt.title(r'$V=\log_2(T)$')
# plt.ylim(-50, 120)
# plt.legend(loc='lower right')

# # plot
# T_2 = get_T_2(T)
# plt.plot(range(1,len(T_2)+1),rev_opt6,label='Oracle solution', linestyle='--')
# plt.plot(range(1,len(T_2)+1),rev_alg6,label=r'Algorithm 2', marker='.')
# # plt.plot(range(1,len(T_2)+1),rev_alg6_1,label=r'$T_0=T^{\frac{1}{2}}$', marker='.')
# # plt.plot(range(1,len(T_2)+1),rev_alg6_2,label=r'$T_0=T^{\frac{2}{3}}$', marker='.')
# plt.xlabel(r'$log_2(T)$')
# plt.ylabel('avg profit')
# # plt.title(r'Nonstationary Demand Functions with $b_i(t) = -(5+sin(\frac{V\pi t}{2T}))$')
# plt.title(r'$V=T^{0.2}$')
# plt.ylim(-75, 120)
# plt.legend(loc='lower right')

# # plot
# T_2 = get_T_2(T)
# plt.plot(range(1,len(T_2)+1),rev_opt7,label='Oracle solution', linestyle='--')
# plt.plot(range(1,len(T_2)+1),rev_alg7,label='Algorithm 2', marker='.')
# # plt.plot(range(1,len(T_2)+1),rev_alg7_1,label=r'$T_0=T^{\frac{1}{2}}$', marker='.')
# # plt.plot(range(1,len(T_2)+1),rev_alg7_2,label=r'$T_0=T^{\frac{2}{3}}$', marker='.')
# plt.xlabel(r'$log_2(T)$')
# plt.ylabel('avg profit')
# # plt.title(r'Nonstationary Demand Functions with $b_i(t) = -(5+sin(\frac{V\pi t}{2T}))$')
# plt.title(r'$V=T^{0.5}$')
# plt.ylim(-75, 120)
# plt.legend(loc='lower right')

# # store
# with open('results/output_2_TRUE_.pkl', 'wb') as f:
#     pickle.dump([rev_alg5, rev_opt5, rev_alg6, rev_opt6, rev_alg7, rev_opt7], f)
# f.close()

# # store
# with open('results/output_2_TRUE_final.pkl', 'wb') as f:
#     pickle.dump([rev_alg5, rev_alg5_1, rev_alg5_2, rev_opt5, 
#                  rev_alg6, rev_alg6_1, rev_alg6_2, rev_opt6, 
#                  rev_alg7, rev_alg7_1, rev_alg7_2, rev_opt7], f)
# f.close()


# # reload
# with open('results/output_2_TRUE_.pkl', 'rb') as f:
#     rev_alg5, rev_opt5, rev_alg6, rev_opt6, rev_alg7, rev_opt7 = pickle.load(f)
# f.close()

# reload
with open('results/output_2_TRUE_final.pkl', 'rb') as f:
    rev_alg5, rev_alg5_1, rev_alg5_2, rev_opt5, rev_alg6, rev_alg6_1, rev_alg6_2, rev_opt6, rev_alg7, rev_alg7_1, rev_alg7_2, rev_opt7 = pickle.load(f)
f.close()

## Compare with Explore-then-exploit Heuristic

In [ ]:
# # compute
# # compute H=5, m=3
# # compute Exp-exp
# rev_new_1, rev_opt_1 = learning_regret(m, M, T, H, p_u, REGRET=False, ALG=2, LOG_T=True, UNI_ALOC=False)
# rev_new_2, rev_opt_2 = learning_regret(m, 100, T, H, p_u, REGRET=False, ALG=3, LOG_T=True, UNI_ALOC=False)
# rev_new_3, rev_opt_3 = learning_regret(m, 100, T, H, p_u, REGRET=False, ALG=3, LOG_T=True, UNI_ALOC=False, FRAC=2)

# # compute H=5, m=3
# # compute Exp-exp
# rev_new_4, rev_opt_4 = learning_regret(m, M, T, H, p_u, REGRET=False, ALG=2, LOG_T=True, UNI_ALOC=True)
# rev_new_5, rev_opt_5 = learning_regret(m, 100, T, H, p_u, REGRET=False, ALG=3, LOG_T=True, UNI_ALOC=True)
# rev_new_6, rev_opt_6 = learning_regret(m, 100, T, H, p_u, REGRET=False, ALG=3, LOG_T=True, UNI_ALOC=True, FRAC=2)

# # compute H=5, m=3
# # compute Exp-exp
# rev_new_7 = learning_regret(m, M, T, H, p_u, REGRET=True, ALG=2, LOG_T=True, UNI_ALOC=False)
# rev_new_8 = learning_regret(m, 100, T, H, p_u, REGRET=True, ALG=3, LOG_T=True, UNI_ALOC=False)
# rev_new_9 = learning_regret(m, 100, T, H, p_u, REGRET=True, ALG=3, LOG_T=True, UNI_ALOC=False, FRAC=2)

# # compute H=5, m=3
# # compute Exp-exp
# rev_new_10 = learning_regret(m, M, T, H, p_u, REGRET=True, ALG=2, LOG_T=True, UNI_ALOC=True)
# rev_new_11 = learning_regret(m, 100, T, H, p_u, REGRET=True, ALG=3, LOG_T=True, UNI_ALOC=True)
# rev_new_12 = learning_regret(m, 100, T, H, p_u, REGRET=True, ALG=3, LOG_T=True, UNI_ALOC=True, FRAC=2)

# # plot
# T_2 = get_T_2(T)
# plt.plot(range(1,len(T_2)+1),rev_new_1,label='Algorithm 2', marker='.')
# plt.plot(range(1,len(T_2)+1),rev_opt_4,label='Oracle solution', linestyle='--') 
# plt.plot(range(1,len(T_2)+1),rev_new_2,label=r'Exp-Exp with $T_0 = T^{\frac{1}{2}}$', marker='.')
# plt.plot(range(1,len(T_2)+1),rev_new_3,label=r'Exp-Exp with $T_0 = T^{\frac{2}{3}}$', marker='.')
# plt.xlabel(r'$log_2(T)$')
# plt.ylabel('avg profit')
# plt.title('Average profit for different time horizons (optimal allocation)')
# plt.legend()

# T_2 = get_T_2(T)
# plt.plot(range(1,len(T_2)+1),rev_new_4,label='Algorithm 2', marker='.')
# plt.plot(range(1,len(T_2)+1),rev_opt_4,label='Oracle solution', linestyle='--') 
# plt.plot(range(1,len(T_2)+1),rev_new_5,label=r'Exp-Exp with $T_0 = T^{\frac{1}{2}}$', marker='.')
# plt.plot(range(1,len(T_2)+1),rev_new_6,label=r'Exp-Exp with $T_0 = T^{\frac{2}{3}}$', marker='.')
# plt.xlabel(r'$log_2(T)$')
# plt.ylabel('avg profit')
# plt.title('Average profit for different time horizons (uniform allocation)')
# plt.legend()

# T_2 = get_T_2(T)
# plt.plot(range(1,len(T_2)+1),[rev_new_7[i]/T_2[i] for i in range(len(T_2))],label='Algorithm 2', marker='.')
# plt.plot(range(1,len(T_2)+1),[rev_new_8[i]/T_2[i] for i in range(len(T_2))],label=r'Exp-Exp with $T_0 = T^{\frac{1}{2}}$', marker='.')
# plt.plot(range(1,len(T_2)+1),[rev_new_9[i]/T_2[i] for i in range(len(T_2))],label=r'Exp-Exp with $T_0 = T^{\frac{2}{3}}$', marker='.')
# plt.xlabel(r'$log_2(T)$')
# plt.ylabel('avg learning regret')
# plt.title('Average learning regret for different time horizons (optimal allocation)')
# plt.legend()

# T_2 = get_T_2(T)
# plt.plot(range(1,len(T_2)+1),[rev_new_10[i]/T_2[i] for i in range(len(T_2))],label='Algorithm 2', marker='.')
# plt.plot(range(1,len(T_2)+1),[rev_new_11[i]/T_2[i] for i in range(len(T_2))],label=r'Exp-Exp with $T_0 = T^{\frac{1}{2}}$', marker='.')
# plt.plot(range(1,len(T_2)+1),[rev_new_12[i]/T_2[i] for i in range(len(T_2))],label=r'Exp-Exp with $T_0 = T^{\frac{2}{3}}$', marker='.')
# plt.xlabel(r'$log_2(T)$')
# plt.ylabel('avg learning regret')
# plt.title('Average learning regret for different time horizons (uniform allocation)')
# plt.legend()

# # store
# variables = [rev_new_1, rev_opt_1, rev_new_2, rev_opt_2, rev_new_3, rev_opt_3, 
#              rev_new_4, rev_opt_4, rev_new_5, rev_opt_5, rev_new_6, rev_opt_6,
#              rev_new_7, rev_new_8, rev_new_9, rev_new_10, rev_new_11, rev_new_12
#             ]
# with open('results/output_3_2.pkl', 'wb') as f:
#     pickle.dump(variables, f)

# # reload
# with open('results/output_3_.pkl', 'rb') as f:
#     rev_new_1, rev_opt_1, rev_new_2, rev_opt_2, rev_new_3, rev_opt_3, rev_new_4, rev_opt_4, rev_new_5, rev_opt_5, rev_new_6, rev_opt_6, rev_new_7, rev_new_8, rev_new_9, rev_new_10, rev_new_11, rev_new_12 = pickle.load(f)
# f.close()

In [ ]:
# print(compute_relative_profit_loss(rev_opt_1, rev_new_1)[9:12])

# print(compute_relative_profit_loss(rev_opt_2, rev_new_2)[9:12])

# print(compute_relative_profit_loss(rev_opt_3, rev_new_3)[9:12])

In [ ]:
# # compute H=20, m=5
# # compute Exp-exp
# rev_alg8, rev_opt8 = learning_regret(m, M, T, H, p_u, REGRET=False, ALG=2, LOG_T=True, UNI_ALOC=True)

# rev_alg9, rev_opt9 = learning_regret(m, M, T, H, p_u, REGRET=False, ALG=3, LOG_T=True, UNI_ALOC=True)

# # compute H=20, m=5
# # compute Exp-exp
# rev_alg10, rev_opt10 = learning_regret(m, M, T, H, p_u, REGRET=False, ALG=2, LOG_T=True)

# rev_alg11, rev_opt11 = learning_regret(m, M, T, H, p_u, REGRET=False, ALG=3, LOG_T=True)

# # compute H=10, m=3 UNIFORM
# # compute Exp-exp
# rev_alg12, rev_opt12 = learning_regret(m, M, T, H, p_u, REGRET=False, ALG=2, LOG_T=True, UNI_ALOC=True)

# rev_alg13, rev_opt13 = learning_regret(m, M, T, H, p_u, REGRET=False, ALG=3, LOG_T=True, UNI_ALOC=True)

# # plot
# T_2 = get_T_2(T)
# plt.plot(range(1,len(T_2)+1),rev_alg2,label='Algorithm 2', marker='.')
# plt.plot(range(1,len(T_2)+1),rev_opt1,label='Oracle solution', linestyle='--') 
# plt.plot(range(1,len(T_2)+1),rev_alg1,label='Exp-Exp', marker='.')
# plt.xlabel(r'$log_2(T)$')
# plt.ylabel('avg profit')
# plt.title('Stationary demand and optimal allocation')
# plt.legend()

# # plot
# T_2 = get_T_2(T)
# plt.plot(range(1,len(T_2)+1),rev_alg12,label='Algorithm 2', marker='.')
# plt.plot(range(1,len(T_2)+1),rev_opt13,label='Oracle solution', linestyle='--') 
# plt.plot(range(1,len(T_2)+1),rev_alg13,label='Exp-Exp', marker='.')
# plt.xlabel(r'$log_2(T)$')
# plt.ylabel('avg profit')
# plt.title('Stationary demand and uniform allocation')
# plt.legend()

# # plot
# T_2 = get_T_2(T)
# plt.plot(range(1,len(T_2)+1),rev_alg8,label='Algorithm 2', marker='.')
# plt.plot(range(1,len(T_2)+1),rev_opt8,label='Oracle solution', linestyle='--') 
# plt.plot(range(1,len(T_2)+1),rev_alg9,label='Exp-Exp', marker='.')
# plt.xlabel(r'$log_2(T)$')
# plt.ylabel('avg profit')
# plt.title('Stationary demand and optimal allocation')
# plt.legend()

# # plot
# T_2 = get_T_2(T)
# plt.plot(range(1,len(T_2)+1),rev_alg10,label='Algorithm 2', marker='.')
# plt.plot(range(1,len(T_2)+1),rev_opt10,label='Oracle solution', linestyle='--') 
# plt.plot(range(1,len(T_2)+1),rev_alg11,label='Exp-Exp', marker='.')
# plt.xlabel(r'$log_2(T)$')
# plt.ylabel('avg profit')
# plt.title('Stationary demand and uniform allocation')
# plt.legend()

# # store
# with open('results/output_3.pkl', 'wb') as f:
#     pickle.dump([rev_alg2, rev_opt2, rev_alg1, rev_opt1,
#                 rev_alg12, rev_opt12, rev_alg13, rev_opt13,
#                 rev_alg8, rev_opt8, rev_alg9, rev_opt9,
#                 rev_alg10, rev_opt10, rev_alg11, rev_opt11], f)
# f.close()

# reload
with open('results/output_3.pkl', 'rb') as f:
    rev_alg2, rev_opt2, rev_alg1, rev_opt1, rev_alg12, rev_opt12, rev_alg13, rev_opt13, rev_alg8, rev_opt8, rev_alg9, rev_opt9, rev_alg10, rev_opt10, rev_alg11, rev_opt11 = pickle.load(f)
f.close()

## More figures

In [ ]:
# # compute
# # compute Total regret
# reg_alg1, Treg_alg1 = learning_regret(m, 100, T, H, p_u, ALG=2, LOG_T=True, REGRET=True, TOTAL_REGRET=True)

# # h=20, m=5
# reg_alg2 = learning_regret(m, M, T, H, p_u, ALG=2, LOG_T=True, REGRET=True)

# # h=20, m=5
# rev_alg2_4, rev_opt2_4 = learning_regret(m, M, T, H, p_u, ALG=2, LOG_T=True, REGRET=False)

# # plot
# T_2 = get_T_2(T)
# plt.plot(range(1,len(T_2)+1), Treg_alg1, label='total regret', marker='.')
# plt.xlabel(r'$log_2(T)$')
# plt.ylabel('Total regret')
# plt.title('Total regret with optimal allocation')
# plt.legend()

# # plot
# T_2 = get_T_2(T)
# plt.plot(range(1,len(T_2)+1), reg_alg1, label='learning regret', marker='.')
# plt.xlabel(r'$log_2(T)$')
# plt.ylabel('Learning regret')
# plt.title('Learning regret with optimal allocation')
# plt.legend()

# # plot
# T_2 = get_T_2(T)
# plt.plot(range(1,len(T_2)+1), [reg_alg2[i]/T_2[i] for i in range(len(T_2))], label='learning regret', marker='.')
# plt.xlabel(r'$log_2(T)$')
# plt.ylabel('learning regret')
# plt.title('Average learning regret with optimal allocation')
# plt.legend()

# # plot
# T_2 = get_T_2(T)
# plt.plot(range(1,len(T_2)+1),rev_alg2_4,label='Algorithm 2', marker='.')
# plt.plot(range(1,len(T_2)+1),rev_opt2_4,label='Oracle solution', linestyle='--') 
# plt.xlabel(r'$log_2(T)$')
# plt.ylabel('avg profit')
# plt.title('Average profit for algorithm 2')
# plt.legend()

# # store
# with open('results/output_4.pkl', 'wb') as f:
#     pickle.dump([reg_alg1, Treg_alg1, reg_alg2, rev_alg2_4, rev_opt2_4], f)
# f.close()

# reload
with open('results/output_4.pkl', 'rb') as f:   
    reg_alg1, Treg_alg1, reg_alg2, rev_alg2_4, rev_opt2_4 = pickle.load(f)
f.close()

## Phase Transition

In [ ]:
# # # compute
# m_range = np.arange(1, H)

# Treg5 = [[0]]
# reg5 = [[0]]
# for m in m_range:
#     reg_epoch, Treg_epoch = learning_regret(m, 1, T, H=H, p_u=p_u, ALG=2, LOG_T=True, REGRET=True, TOTAL_REGRET=True)
#     print(reg_epoch[-1] - reg5[-1][-1])
#     print(Treg_epoch[-1] - Treg5[-1][-1])
#     Treg5.append(Treg_epoch)
#     reg5.append(reg_epoch)

# Treg6 = []
# reg6 = []
# for m in m_range:
#     _, Treg_epoch = learning_regret(m, 1, T, H, p_u, ALG=2, REGRET=True, LOG_T=True, TOTAL_REGRET=True, UNI_ALOC=True)
#     print(Treg_epoch)
#     Treg6.append(Treg_epoch)
#     reg6.append(_)

# # H=10, m=5
# m_range = np.arange(1, H)

# Treg7 = []
# for m in m_range:
#    _, Treg_epoch = learning_regret(m, 3, T, H=H, p_u=p_u, ALG=2, LOG_T=True, REGRET=True, TOTAL_REGRET=True)
#    print(Treg_epoch)
#    Treg7.append(Treg_epoch)

# Treg8 = []
# for m in m_range:
#    _, Treg_epoch = learning_regret(m, 3, T, H, p_u, ALG=2, REGRET=True, LOG_T=True, TOTAL_REGRET=True, UNI_ALOC=True)
#    print(Treg_epoch)
#    Treg8.append(Treg_epoch)

# # plot
# T_2 = get_T_2(T)
# plt.plot(m_range, [reg6[i][-1] for i in range(len(reg6))], label=r'uniform allocation', marker='.')
# plt.plot(m_range, [reg5[i][-1] for i in range(len(reg5))], label=r'optimal allocation', marker='.')
# plt.xlabel(r'$m$')
# plt.ylabel('learning regret')
# plt.title(r'Learning regret of different allocations ($T=2^{12}$)')
# plt.legend()

# # plot
# T_2 = get_T_2(T)
# plt.plot(m_range, [Treg6[i][-1] for i in range(len(Treg6))], label=r'uniform allocation', marker='.')
# plt.plot(m_range, [Treg5[i][-1] for i in range(len(Treg5))], label=r'optimal allocation', marker='.')
# plt.xlabel(r'$m$')
# plt.ylabel('regret')
# plt.title('Regret of different allocations')
# plt.legend()
# # plt.ylim(8000,30000)

# # plot
# T_2 = get_T_2(T)
# plt.plot(m_range, [Treg6[i][9] for i in range(len(Treg6))], label=r'$T=2^{10}$', marker='.')
# plt.plot(m_range, [Treg6[i][10] for i in range(len(Treg6))], label=r'$T=2^{11}$', marker='.')
# plt.plot(m_range, [Treg6[i][11] for i in range(len(Treg6))], label=r'$T=2^{12}$', marker='.')
# plt.xlabel(r'$m$')
# plt.ylabel('Total regret')
# plt.title('Total regret with uniform allocation')
# plt.legend()

# # plot
# T_2 = get_T_2(T)
# # plt.plot(m_range, [Treg7[i][9] for i in range(len(Treg7))], label=r'$T=2^{10}$', marker='.')
# # plt.plot(m_range, [Treg7[i][10] for i in range(len(Treg7))], label=r'$T=2^{11}$', marker='.')
# plt.plot(m_range, [Treg7[i][11] for i in range(len(Treg7))], label=r'$T=2^{12}$', marker='.')
# plt.xlabel(r'$m$')
# plt.ylabel('Total regret')
# plt.title('Total regret with optimal allocation')
# plt.legend()

# # plot
# T_2 = get_T_2(T)
# plt.plot(m_range, [Treg8[i][9] for i in range(len(Treg8))], label=r'$T=2^{10}$', marker='.')
# plt.plot(m_range, [Treg8[i][10] for i in range(len(Treg8))], label=r'$T=2^{11}$', marker='.')
# plt.plot(m_range, [Treg8[i][11] for i in range(len(Treg8))], label=r'$T=2^{12}$', marker='.')
# plt.xlabel(r'$m$')
# plt.ylabel('Total regret')
# plt.title('Total regret with uniform allocation')
# plt.legend()

# # store
# with open('results/output_5_tmp.pkl', 'wb') as f:
#     pickle.dump([Treg5, Treg6, reg5, reg6], f)
# f.close()

# reload
with open('results/output_5_.pkl', 'rb') as f:   
    Treg5, Treg6, Treg7, Treg8 = pickle.load(f)
f.close()

## Compare uniform allocation and optimal allocation

In [ ]:
# # compute
# # H=10
# output_6 = []
# for m in np.arange(1,H):
#     new_reg = learning_regret(m, 1, T, H=H, p_u=p_u, ALG=2, MODIFIED=2, REGRET=True)
#     print(new_reg)
#     output_6.append(new_reg)

# # H=10
# output_7 = []
# for m in np.arange(1,H):
#     new_reg = learning_regret(m, 1, T, H=H, p_u=p_u, ALG=2, MODIFIED=2, REGRET=True, UNI_ALOC=True)
#     print(new_reg)
#     output_7.append(new_reg)

# plt.plot(range(1,len(output_6)+1), output_6, label='optimal allocation',marker='.')
# # plt.plot(range(1,len(output_7)+1), output_7, label='uniform alllocation',marker='.')
# plt.xlabel('m')
# plt.ylabel('Learning Regret')
# # plt.text(x = 7, y = 1500, s = r'$T = 2^{7}, H = 15, a = 20, b = -5$')
# plt.title('Learning Regret of Different Allocation')
# plt.legend()

# plt.plot(range(1,len(output_6)+1), output_6, label='optimal allocation',marker='.')
# plt.plot(range(1,len(output_7)+1), output_7, label='uniform alllocation',marker='.')
# plt.xlabel('m')
# plt.ylabel('Learning Regret')
# # plt.text(x = 7, y = 1500, s = r'$T = 2^{7}, H = 15, a = 20, b = -5$')
# plt.title('Learning Regret of Different Allocation')
# plt.legend()

# # store
# with open('results/output_6_tmp.pkl', 'wb') as f:
#     pickle.dump([output_6, output_7], f)
# f.close()

# reload
with open('results/output_6_tmp.pkl', 'rb') as f:   
    output_6, output_7 = pickle.load(f)
f.close()

## Compare average total regret for m = H − 3, H − 2, H − 1

In [ ]:
# # compute
# # H=10
# output_8 = []
# output_9 = []

# T_2 = get_T_2(T)

# for m in np.arange(H-3,H):
#     new_reg, Tnew_reg = learning_regret(m, 3, T, H=H, p_u=p_u, ALG=2, REGRET=True, LOG_T=True, TOTAL_REGRET=True)
#     new_reg = [new_reg[i] / T_2[i] for i in range(len(T_2))]
#     Tnew_reg = [Tnew_reg[i] / T_2[i] for i in range(len(T_2))]
#     print(f'm = {m} has finished...')
#     output_8.append(new_reg)
#     output_9.append(Tnew_reg)

# output_8_ = []
# output_9_ = []
# for m in np.arange(H-3,H):
#     new_reg, Tnew_reg = learning_regret(m, 3, T, H=H, p_u=p_u, ALG=2, REGRET=True, LOG_T=True, TOTAL_REGRET=True, UNI_ALOC=True)
#     new_reg = [new_reg[i] / T_2[i] for i in range(len(T_2))]
#     Tnew_reg = [Tnew_reg[i] / T_2[i] for i in range(len(T_2))]
#     print(f'm = {m} has finished...')
#     output_8_.append(new_reg)
#     output_9_.append(Tnew_reg)

# # plot
# plt.plot(range(10,len(output_9[0])+1), output_9[0][9:], label='m=H-3',marker='.')
# plt.plot(range(10,len(output_9[1])+1), output_9[1][9:], label='m=H-2',marker='.')
# plt.plot(range(10,len(output_9[2])+1), output_9[2][9:], label='m=H-1',marker='.')
# plt.xlabel(r'$log_2(T)$')
# plt.ylabel('avg regret')
# plt.title('Average regret for different pricing flexibilities')
# plt.legend()

# plot
plt.plot(range(1,len(output_9[0])+1), minus(output_9_[0],output_9[0]), label='m=H-3',marker='.')
plt.plot(range(1,len(output_9[1])+1), minus(output_9_[1],output_9[1]), label='m=H-2',marker='.')
plt.plot(range(1,len(output_9[2])+1), minus(output_9_[2],output_9[2]), label='m=H-1',marker='.')
plt.xlabel(r'$log_2(T)$')
plt.ylabel('Average Total Regret')
plt.title('Average total regret of optimal allocation')
plt.legend()

# m is x axis, solve 2 DP: m = H-1; and m; minus
# first term

plt.plot(range(10,len(output_9_[0])+1), output_9_[0][9:], label='m=H-3',marker='.')
plt.plot(range(10,len(output_9_[1])+1), output_9_[1][9:], label='m=H-2',marker='.')
plt.plot(range(10,len(output_9_[2])+1), output_9_[2][9:], label='m=H-1',marker='.')
plt.xlabel(r'$log_2(T)$')
plt.ylabel('Average Total Regret')
plt.title('Average total regret of uniform allocation')
plt.legend()

# # store
# with open('results/output_7_tmp.pkl', 'wb') as f:
#     pickle.dump([output_8, output_9, output_8_, output_9_], f)
# f.close()

# reload
with open('results/output_7_tmp.pkl', 'rb') as f:   
    output_8, output_9, output_8_, output_9_ = pickle.load(f)
f.close()

In [ ]:
# # compute
# # H=10
# output_8 = []
# output_9 = []

# T_2 = get_T_2(T)

# for m in np.arange(H-3,H):
#     new_reg, Tnew_reg = learning_regret(m, 3, T, H=H, p_u=p_u, ALG=2, REGRET=True, LOG_T=True, TOTAL_REGRET=True)
#     new_reg = [new_reg[i] / T_2[i] for i in range(len(T_2))]
#     Tnew_reg = [Tnew_reg[i] / T_2[i] for i in range(len(T_2))]
#     print(f'm = {m} has finished...')
#     output_8.append(new_reg)
#     output_9.append(Tnew_reg)

# output_8_ = []
# output_9_ = []
# for m in np.arange(H-3,H):
#     new_reg, Tnew_reg = learning_regret(m, 3, T, H=H, p_u=p_u, ALG=2, REGRET=True, LOG_T=True, TOTAL_REGRET=True, UNI_ALOC=True)
#     new_reg = [new_reg[i] / T_2[i] for i in range(len(T_2))]
#     Tnew_reg = [Tnew_reg[i] / T_2[i] for i in range(len(T_2))]
#     print(f'm = {m} has finished...')
#     output_8_.append(new_reg)
#     output_9_.append(Tnew_reg)

# # plot
# plt.plot(range(10,len(output_9[0])+1), output_9[0][9:], label='m=H-3',marker='.')
# plt.plot(range(10,len(output_9[1])+1), output_9[1][9:], label='m=H-2',marker='.')
# plt.plot(range(10,len(output_9[2])+1), output_9[2][9:], label='m=H-1',marker='.')
# plt.xlabel(r'$log_2(T)$')
# plt.ylabel('Average Total Regret')
# plt.title('Average total regret of optimal allocation')
# plt.legend()

# plt.plot(range(10,len(output_9_[0])+1), output_9_[0][9:], label='m=H-3',marker='.')
# plt.plot(range(10,len(output_9_[1])+1), output_9_[1][9:], label='m=H-2',marker='.')
# plt.plot(range(10,len(output_9_[2])+1), output_9_[2][9:], label='m=H-1',marker='.')
# plt.xlabel(r'$log_2(T)$')
# plt.ylabel('Average Total Regret')
# plt.title('Average total regret of uniform allocation')
# plt.legend()

# T_2 = get_T_2(T)
# # plot
# plt.plot([2**i for i in range(1,len(output_9[0])+1)], [output_9[0][i] * T_2[i] for i in range(len(T_2))], label='m=H-3',marker='.')
# plt.plot([2**i for i in range(1,len(output_9[1])+1)], [output_9[1][i] * T_2[i] for i in range(len(T_2))], label='m=H-2',marker='.')
# plt.plot([2**i for i in range(1,len(output_9[2])+1)], [output_9[2][i] * T_2[i] for i in range(len(T_2))], label='m=H-1',marker='.')
# plt.xlabel(r'$T$')
# plt.ylabel('regret')
# plt.title('Regret for different pricing flexibilities')
# plt.legend()

# # store
# with open('results/output_7_tmp.pkl', 'wb') as f:
#     pickle.dump([output_8, output_9, output_8_, output_9_], f)
# f.close()


# reload
with open('results/output_7_tmp.pkl', 'rb') as f:   
    output_8, output_9, output_8_, output_9_ = pickle.load(f)
f.close()

In [ ]:
# # compute
# # H=10
# output_8 = []
# output_9 = []

# T_2 = np.arange(1000,4500,500)
# T_2.tolist()

# for m in np.arange(H-3,H):
#     new_reg, Tnew_reg = learning_regret(m, 3, T, H=H, p_u=p_u, ALG=2, REGRET=True, LOG_T=True, TOTAL_REGRET=True)
#     new_reg = [new_reg[i] / T_2[i] for i in range(len(T_2))]
#     Tnew_reg = [Tnew_reg[i] / T_2[i] for i in range(len(T_2))]
#     print(f'm = {m} has finished...')
#     output_8.append(new_reg)
#     output_9.append(Tnew_reg)

# # plot
# plt.plot(T_2, [output_9[0][i] * T_2[i] for i in range(len(T_2))], label='m=H-3',marker='.')
# plt.plot(T_2, [output_9[1][i] * T_2[i] for i in range(len(T_2))], label='m=H-2',marker='.')
# plt.plot(T_2, [output_9[2][i] * T_2[i] for i in range(len(T_2))], label='m=H-1',marker='.')
# plt.xlabel(r'$T$')
# plt.ylabel('regret')
# plt.title('Regret for different pricing flexibilities')
# plt.legend()

# # store
# with open('results/output_7_1000_4000.pkl', 'wb') as f:
#     pickle.dump([output_8, output_9], f)
# f.close()

# reload
with open('results/output_7_1000_4000.pkl', 'rb') as f:   
    output_8, output_9 = pickle.load(f)
f.close()

## Changing Demand

In [ ]:
# # compute
# # compute H=5, m=3
# # compute increasing
# rev_ch_1, rev_cop_1 = learning_regret(m, 100, T, H, p_u, REGRET=False, ALG=2, LOG_T=True, UNI_ALOC=False)
# rev_ch_2, rev_cop_2 = learning_regret(m, 100, T, H, p_u, REGRET=False, ALG=3, LOG_T=True, UNI_ALOC=False)
# rev_ch_3, rev_cop_3 = learning_regret(m, 100, T, H, p_u, REGRET=False, ALG=3, LOG_T=True, UNI_ALOC=False, FRAC=2)


# # compute decreasing
# rev_ch_4, rev_cop_4 = learning_regret(m, 100, T, H, p_u, REGRET=False, ALG=2, LOG_T=True, UNI_ALOC=False)
# rev_ch_5, rev_cop_5 = learning_regret(m, 100, T, H, p_u, REGRET=False, ALG=3, LOG_T=True, UNI_ALOC=False)
# rev_ch_6, rev_cop_6 = learning_regret(m, 100, T, H, p_u, REGRET=False, ALG=3, LOG_T=True, UNI_ALOC=False, FRAC=2)


# # compute increasing then decreasing
# rev_ch_7, rev_cop_7 = learning_regret(m, 100, T, H, p_u, REGRET=False, ALG=2, LOG_T=True, UNI_ALOC=False)
# rev_ch_8, rev_cop_8 = learning_regret(m, 100, T, H, p_u, REGRET=False, ALG=3, LOG_T=True, UNI_ALOC=False)
# rev_ch_9, rev_cop_9 = learning_regret(m, 100, T, H, p_u, REGRET=False, ALG=3, LOG_T=True, UNI_ALOC=False, FRAC=2)


# # compute non-monotone
# rev_ch_10, rev_cop_10 = learning_regret(m, 100, T, H, p_u, REGRET=False, ALG=2, LOG_T=True, UNI_ALOC=False)
# rev_ch_11, rev_cop_11 = learning_regret(m, 100, T, H, p_u, REGRET=False, ALG=3, LOG_T=True, UNI_ALOC=False)
# rev_ch_12, rev_cop_12 = learning_regret(m, 100, T, H, p_u, REGRET=False, ALG=3, LOG_T=True, UNI_ALOC=False, FRAC=2)

# # plot
# T_2 = get_T_2(T)
# plt.plot(range(1,len(T_2)+1),rev_ch_1,label='Algorithm 2', marker='.')
# plt.plot(range(1,len(T_2)+1),rev_cop_1,label='Oracle solution', linestyle='--') 
# plt.plot(range(1,len(T_2)+1),rev_ch_2,label=r'Exp-Exp with $T_0 = T^{\frac{1}{2}}$', marker='.')
# plt.plot(range(1,len(T_2)+1),rev_ch_3,label=r'Exp-Exp with $T_0 = T^{\frac{2}{3}}$', marker='.')
# plt.xlabel(r'$log_2(T)$')
# plt.ylabel('avg profit')
# plt.title('Increasing')
# plt.legend()

# T_2 = get_T_2(T)
# plt.plot(range(1,len(T_2)+1),rev_ch_4,label='Algorithm 2', marker='.')
# plt.plot(range(1,len(T_2)+1),rev_cop_4,label='Oracle solution', linestyle='--') 
# plt.plot(range(1,len(T_2)+1),rev_ch_5,label=r'Exp-Exp with $T_0 = T^{\frac{1}{2}}$', marker='.')
# plt.plot(range(1,len(T_2)+1),rev_ch_6,label=r'Exp-Exp with $T_0 = T^{\frac{2}{3}}$', marker='.')
# plt.xlabel(r'$log_2(T)$')
# plt.ylabel('avg profit')
# plt.title('Decreasing')
# plt.legend()

# T_2 = get_T_2(T)
# plt.plot(range(1,len(T_2)+1),rev_ch_7,label='Algorithm 2', marker='.')
# plt.plot(range(1,len(T_2)+1),rev_cop_7,label='Oracle solution', linestyle='--') 
# plt.plot(range(1,len(T_2)+1),rev_ch_8,label=r'Exp-Exp with $T_0 = T^{\frac{1}{2}}$', marker='.')
# plt.plot(range(1,len(T_2)+1),rev_ch_9,label=r'Exp-Exp with $T_0 = T^{\frac{2}{3}}$', marker='.')
# plt.xlabel(r'$log_2(T)$')
# plt.ylabel('avg profit')
# plt.title('Increasing then decreasing')
# plt.legend()

# plt.plot(range(1,len(T_2)+1),rev_ch_10,label='Algorithm 2', marker='.')
# plt.plot(range(1,len(T_2)+1),rev_cop_10,label='Oracle solution', linestyle='--') 
# plt.plot(range(1,len(T_2)+1),rev_ch_11,label=r'Exp-Exp with $T_0 = T^{\frac{1}{2}}$', marker='.')
# plt.plot(range(1,len(T_2)+1),rev_ch_12,label=r'Exp-Exp with $T_0 = T^{\frac{2}{3}}$', marker='.')
# plt.xlabel(r'$log_2(T)$')
# plt.ylabel('avg profit')
# plt.title('Nonmonotone')
# plt.legend()

# # store
# with open('results/output_change_demand.pkl', 'wb') as f:
#     pickle.dump([rev_ch_1, rev_cop_1, rev_ch_2, rev_cop_2, rev_ch_3, rev_cop_3, rev_ch_4, rev_cop_4, rev_ch_5, rev_cop_5,rev_ch_6, rev_cop_6
#                 ,rev_ch_7, rev_cop_7, rev_ch_8, rev_cop_8, rev_ch_9, rev_cop_9, rev_ch_10, rev_cop_10, rev_ch_11, rev_cop_11, rev_ch_12, rev_cop_12], f)
# f.close()

# reload
with open('results/output_change_demand.pkl', 'rb') as f:   
    rev_ch_1, rev_cop_1, rev_ch_2, rev_cop_2, rev_ch_3, rev_cop_3, rev_ch_4, rev_cop_4, rev_ch_5, rev_cop_5,rev_ch_6, rev_cop_6,rev_ch_7, rev_cop_7, rev_ch_8, rev_cop_8, rev_ch_9, rev_cop_9, rev_ch_10, rev_cop_10, rev_ch_11, rev_cop_11, rev_ch_12, rev_cop_12 = pickle.load(f)
f.close()